## 참조자료

https://myungpyo.medium.com/reading-coroutine-official-guide-thoroughly-part-1-98f6e792bd5b

## 코루티 모듈 로딩 

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.1")

## 설치 확인 

In [2]:
:classpath

Current classpath (14 paths):
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/lib-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/api-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.7.0-dev-1825.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.6.0.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx/kotlinx-coroutines-core/1.6.1/kotlinx-coroutines-core-1.6.1.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx

## 기본 사용 패키지

In [3]:
import kotlinx.coroutines.*


## 디스패처 알아보기

- Dispatcher

Dispatcher는 CoroutineContext를 상속받아 어떤 쓰레드를 이용해서 동작할 것인지 정의해둔 것 입니다.

 

- None : 실행된 코루틴 스코프에게 상속받는다.

 

- Unconfined : 호출 쓰레드에서 실행하지만 멈췄다가 재개할 경우 재개한 쓰레드에서 실행한다.

-> 특정 쓰레드에 국한되지 않는 경우 적합


- Main : 메인 쓰레드에서 실행한다.

-> 안드로이드에서 UI를 바꿀 때 적합, Job이 무거우면 부적합

 

- IO : 네트워크, 디스크 사용 할때 사용합니다. 파일 읽고, 쓰고, 소켓을 읽고, 쓰고 작업을 멈추는것에 최적화되어 있습니다.

 

- Default : Thread Pool에 할당한다. (GlobalScope.launch { }도 마찬가지)

## 컨텍스트를 의한 디스패처 지정하기

In [5]:
fun main() { 
    runBlocking {
        launch {
            println("launch : ${Thread.currentThread().name}")
        }

        launch(Dispatchers.Unconfined) {
            println("launch(Dispatchers.Unconfined) : ${Thread.currentThread().name}")
        }

       //launch(Dispatchers.Main) {   //안드로이드 용
       //    println("launch(Dispatchers.Main) : ${Thread.currentThread().name}")
       //}

        launch(Dispatchers.IO) {
            println("launch(Dispatchers.IO) : ${Thread.currentThread().name}")
        }

        launch(Dispatchers.Default) {
            println("launch(Dispatchers.Default) : ${Thread.currentThread().name}")
        }
    }
}

main()

launch(Dispatchers.Unconfined) : Thread-26
launch(Dispatchers.IO) : DefaultDispatcher-worker-1
launch(Dispatchers.Default) : DefaultDispatcher-worker-2
launch : Thread-26


## 스코프 컨텍스트 

In [6]:
fun main() = runBlocking {                                     // 런블러킹 컨텍스트 확인 
    println("runBlocking : ${Thread.currentThread().name}")    
    println("컨텍스트 : " + coroutineContext)                                      
    println("상태 : " +isActive)                                // 잡상태 확인
    
    val g = GlobalScope.launch {                               // 전역스코프 컨텍스트 확인 
        delay(300)
        println("전역스코프 : ${Thread.currentThread().name}")
        println("컨텍스트 : " +coroutineContext)
        println("상태 : " +isActive)                            // 잡상태 확인 
    }
    println("전역스코프 반환 : " + g)
    val c =CoroutineScope(Dispatchers.Default).launch {        // 코루틴스코프 컨텍스트 확인
        delay(500)
        println("코루틴 스코프 : ${Thread.currentThread().name}")
        println("컨텍스트 : " +coroutineContext)
        println("상태 : " +isActive)                             // 잡상태 확인    
    }
    println("코르틴 스코프 반환 : " + c)
    delay(1000)
}

main()

runBlocking : Thread-30
컨텍스트 : [BlockingCoroutine{Active}@390a8c69, BlockingEventLoop@5be2ec8b]
상태 : true
전역스코프 반환 : StandaloneCoroutine{Active}@609d6216
코르틴 스코프 반환 : StandaloneCoroutine{Active}@724b1be3
전역스코프 : DefaultDispatcher-worker-1
컨텍스트 : [StandaloneCoroutine{Active}@609d6216, Dispatchers.Default]
상태 : true
코루틴 스코프 : DefaultDispatcher-worker-1
컨텍스트 : [StandaloneCoroutine{Active}@724b1be3, Dispatchers.Default]
상태 : true


## 코루틴 컨텍스트 

In [7]:
fun main1() = runBlocking { 
    val l = launch(Dispatchers.Default) {                          // DefaultDispatcher
        println("Default : ${Thread.currentThread().name}")        // 현재 처리 기준
        println("Job : " + coroutineContext[Job.Key])              // 컨텍스트 내의 잡
        println("컨티뉴에션 : "                                       // 컨티뉴에이션 
                + coroutineContext[kotlin.coroutines.ContinuationInterceptor.Key])
    }
    val a = async(Dispatchers.IO) {                                // Dispatcher IO 
        delay(200)
        println("IO  : ${Thread.currentThread().name}")            // 현재 처리 기준 
        println("Job : " + coroutineContext[Job.Key])              // 잡 
        println("컨티뉴에션 : "                                       // 컨티뉴에이션                       
                + coroutineContext[kotlin.coroutines.ContinuationInterceptor.Key])
    }
    val t = launch(newSingleThreadContext("스레드")) {               // 새로운 스레드 처리
        delay(500)
        println("Thread  : newSingleThreadContext: ${Thread.currentThread().name}")
        println("Job : " + coroutineContext[Job.Key])               // 잡 
        println("컨티뉴에션 : "                                        // 컨티뉴에이션
                + coroutineContext[kotlin.coroutines.ContinuationInterceptor.Key])
    }
    
    delay(1000)
    println("반환값 l : " + l)
    println("반환값 a : " + a)
    println("반환값 t : " + t)
}
main1()

Default : DefaultDispatcher-worker-2
Job : StandaloneCoroutine{Active}@5b1b3e7f
컨티뉴에션 : Dispatchers.Default
IO  : DefaultDispatcher-worker-2
Job : DeferredCoroutine{Active}@1d75a9da
컨티뉴에션 : Dispatchers.IO
Thread  : newSingleThreadContext: 스레드
Job : StandaloneCoroutine{Active}@32d7814f
컨티뉴에션 : java.util.concurrent.ScheduledThreadPoolExecutor@1815b50d[Running, pool size = 1, active threads = 1, queued tasks = 0, completed tasks = 1]
반환값 l : StandaloneCoroutine{Completed}@5b1b3e7f
반환값 a : DeferredCoroutine{Completed}@1d75a9da
반환값 t : StandaloneCoroutine{Completed}@32d7814f


##  스레드와 전역 스코프 처리 비교

In [4]:
fun main() {
    Thread(Runnable {                                   // 3개의 러너블 객체를 만들어서 스레드 처리               
        for (i in 0..2) {
            println("쓰레드 처리  : $i "                   // 3개의 쓰레드를 처리 
                    + Thread.currentThread().name)
            Thread.sleep(500L)                          // 스레드 중단 : 현재 스레드 일시중단 
        }
    }).start()                                          // start 메소드로 스레드 시작

    GlobalScope.launch {                                // 코루틴 전역스코프로 처리 
        repeat(3) {                                     // 3개의 일지중단함수 만들고 코루틴 생성 
            println("코루틴처리   : $it " 
                    + Thread.currentThread().name)      // 코루틴 내의 dispatchre로 처리 
            delay(500L)                                 // 일시 중단 다른 작업 수행 
        }
    }
    
    Thread.sleep(3000L)                                 // 어플리케이션 중단 
}

main()

쓰레드 처리  : 0 Thread-15
코루틴처리   : 0 DefaultDispatcher-worker-1
쓰레드 처리  : 1 Thread-15
코루틴처리   : 1 DefaultDispatcher-worker-1
쓰레드 처리  : 2 Thread-15
코루틴처리   : 2 DefaultDispatcher-worker-1


##  전역스코프에 코루틴 계층화하기  

In [5]:
fun main3() {                                              //  
    GlobalScope.launch {                                   // 부모 코루틴을 전역스코프로 생성
        val job = GlobalScope.launch {                     // 내부에 전역스코프 코루틴 생성 
        
            delay(100L)                                    // 일시중단함수 실행  
            println("World! " 
                     + Thread.currentThread().name)        // 코루틴 내부 스레드
        }
        
        println("Hello, " 
                     + Thread.currentThread().name)        // 부모 코루틴 스레드  
        println(job.javaClass)                             // 반환된 잡 클래스 확인 
        job.join()                                         // 내부 코루틴 처리 종료
    } 
    
    Thread.sleep(2000)                                     //  어플리케애선까지 실행되므로 2초 지연
}

main3()

Hello, DefaultDispatcher-worker-1
class kotlinx.coroutines.StandaloneCoroutine
World! DefaultDispatcher-worker-1


## 전역스코프 처리되는 방식 이해하기

In [8]:
fun main() {
    GlobalScope.launch {                              // 전역 코루틴 처리 
        delay(500L)                                   // 현재 작업을 중단해서 다른 스레드 처리
        println("세상에 온것을 환영! " + Thread.currentThread())
    }
    println("코루틴, "  + Thread.currentThread())       // 출력 
                                                      // 스레드를 중단하는 함수처리 없음  

    GlobalScope.launch {                              // 전역 코루틴 처리 
        delay(1000L)                                  // 현재 작업을 중단 
        println("World!" + Thread.currentThread())
    }
    
    Thread.sleep(3000L)                                // 작업을 중단 : 다른 작업 호출 
    println("Hello,"  + Thread.currentThread())        // 출력 
   
}                                                     
main() 

코루틴, Thread[Thread-38,5,main]
세상에 온것을 환영! Thread[DefaultDispatcher-worker-1,5,main]
World!Thread[DefaultDispatcher-worker-1,5,main]
Hello,Thread[Thread-38,5,main]


## 	동등한 레벨로 전역스코프 처리하기 2 

In [10]:
fun main3() {
    println("코루틴 세상, " 
                      + Thread.currentThread().name)     // 처음 출력 
    
    GlobalScope.launch {                                 // 전역 코루틴 처리 
        delay(1000L)                                     // 현재 작업을 중단하고 다른 처리 요청 
        println("세상 ! " 
                      + Thread.currentThread().name)
    }
    GlobalScope.launch {                                 // 전역 코루틴 처리                   
        delay(1000L)                                     // 다른 코루틴 처리 요청 
        println("환영해요! "
                      + Thread.currentThread().name)
    } 
    runBlocking {                                        // 런블럭킹 함수로 코푸틴 생성
        delay(2000L)                                     // 다른 코루틴 처리 요청 
        println("블럭처리, "
                      + Thread.currentThread().name)
    }
    
    println("코루틴 종료 "  
                      + Thread.currentThread().name)     // 출력 
    Thread.sleep(4000L)                                  // 메인 스레드 중단함수 : 전역스포크 고루틴 작업 호출 
}

main3()

코루틴 세상, Thread-46
세상 ! DefaultDispatcher-worker-2
환영해요! DefaultDispatcher-worker-1
블럭처리, Thread-46
코루틴 종료 Thread-46


## 전역스코프 상태 확인

In [14]:
fun main5() = runBlocking {                            // 고루틴 블러킹 처리 
    val job = GlobalScope.launch {                     // 전역스코프에서 잡을 반환 
        delay(1000L)
        println("코루틴 세상 !")
        println("코루틴 활성화 여부 : " + isActive)        // 활성화 여부
    }
    println("환영해요,")
    job.join()                                         // 코루틴 종료 
    println("코루틴 활성화 여부 : " + job.isActive)        // 활성화 여부
    println("코푸틴 정지 여부   : " + job.isCancelled)     // 중단 여부 
    println("코루틴 종료 여부   : " + job.isCompleted)     // 종료 여부
}

main5()

환영해요,
코루틴 세상 !
코루틴 활성화 여부 : true
코루틴 활성화 여부 : false
코푸틴 정지 여부   : false
코루틴 종료 여부   : true


## 전역스코프는 데몬쓰레드

In [9]:
fun main6() = runBlocking {     
    val job =  GlobalScope.launch {                   // 전역스코프 고루틴
        repeat(1000) { i ->                           // 1000개 쓰레드 생성
            println("전역스코프 코루틴 처리 :  $i ...")
            delay(500L)                               // 현재 코루틴을 멈추고 다른 코루틴 작업
        }
    }
   delay(1300L)                                       // 현재 코루틴 멈추고 다른 코루틴 요청 
   job.cancel()                                       // 전역스코프 종료
}

main6()  

전역스코프 코루틴 처리 :  0 ...
전역스코프 코루틴 처리 :  1 ...
전역스코프 코루틴 처리 :  2 ...


## 전역스코프와 코푸틴 스코프 지정 처리

In [10]:
// interface CoroutineScope                            // 코루틴스코프틑 인터페이스 

fun main(): Unit = runBlocking {                       // 런블럭킹 
    
    println("런블럭스코프 : " 
            + Thread.currentThread().name)
    
    GlobalScope                                        // 글로벌 스코프
        .launch(Dispatchers.Default) {
        println("전역스코프 : "
                + Thread.currentThread().name)
    }
    delay(1000L)
    CoroutineScope(Dispatchers.Default)                // 코루틴 스코프 
        .launch {                        
        println("코루틴스코프 1 : " 
                + Thread.currentThread().name)
    }
        
    coroutineScope {                                   // 코푸틴스코프로 지정된 일시중단함수 
        launch(Dispatchers.Default) {                              
            println("코루틴스코프 2 : " 
                   + Thread.currentThread().name)
        }
    }
}

main()

런블럭스코프 : Thread-43
전역스코프 : DefaultDispatcher-worker-1
코루틴스코프 1 : DefaultDispatcher-worker-1
코루틴스코프 2 : DefaultDispatcher-worker-1


##  중단함수 호출  : runBlocling에서 처리해서 스레드를 그냥로 사용

In [14]:
fun main24() = runBlocking {                      // 런블럭 빌드로 코루틴 생성
    val x = doWorld()                             // 일시중단함수 실행 
}

suspend fun doWorld() = coroutineScope {          // 코푸틴스코프로 지정된 일시중단함수 
    launch {                                      // 빌드함수 정의 
        delay(1000L)                              // 1초 지연 
        println("코루틴 세상 ! " 
                + Thread.currentThread().name)
    }
    
    launch(Dispatchers.Default) {                 // 빌드함수 정의 
        delay(500L)                               // 0.5 초 지연 
        println("코틀린 환영 ! " 
                + Thread.currentThread().name)
    }
    
     launch {                                     // 빌드함수 정의 
        println("안녕하세요!!!"                      // 지연없음 
                + Thread.currentThread().name)
    }
}

main24()

안녕하세요!!!Thread-62
코틀린 환영 ! DefaultDispatcher-worker-1
코루틴 세상 ! Thread-62


## 빌더함수로 코루틴 스코프 처리

In [19]:
// suspend fun <R> coroutineScope(                 // 코루틴 일시증단 함수 
// block: suspend CoroutineScope.() -> R)          // 람다표현식을 받아서 코루틴 빌드 
// : R


fun main() = runBlocking {                         // 런블러킹으로 빌더 
    launch {                                       // 코루틴 런치
        delay(200L)
        println("런블러킹 내부 코루틴 처리 : " 
                + Thread.currentThread())
    }

    coroutineScope {                               // 코루틴스코프 정의 
        launch(Dispatchers.Default){               // 코루틴 빌더 
            delay(500L)
            println("코루틴 내부 코루틴 1  : "          // 내부 코루틴 처리 
                    + Thread.currentThread())
        }
        delay(100L)                                 // 지연처리 
        println("코루틴 스코프 처리  : "                // 현재 상태 확인 
                + Thread.currentThread())
    }
    println("런블러킹 종료  : "                         // 최종 확인 
            + Thread.currentThread())
}
main()

코루틴 스코프 처리  : Thread[Thread-85,5,main]
런블러킹 내부 코루틴 처리 : Thread[Thread-85,5,main]
코루틴 내부 코루틴 1  : Thread[DefaultDispatcher-worker-1,5,main]
런블러킹 종료  : Thread[Thread-85,5,main]
